## Рекомендательные системы

##### Входные данные

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [2]:
sessions = pd.read_csv('./coursera_sessions_train.txt',delimiter=';', header=None,names=['viewed','bought'])

In [3]:
sessions.info()
sessions.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
viewed    50000 non-null object
bought    3608 non-null object
dtypes: object(2)
memory usage: 781.3+ KB


,viewed,bought
0,"0,1,2,3,4,5",NaN
1,"9,10,11,9,11,12,9,11",NaN
2,"16,17,18,19,20,21",NaN
3,"24,25,26,27,24",NaN
4,"34,35,36,34,37,35,36,37,38,39,38,39",NaN
5,42,NaN
6,"47,48,49",NaN
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63"
8,"71,72,73,74",NaN
9,"76,77,78",NaN


##### Важно:

    Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
    Если товар не встречался в обучающей выборке, его популярность равна 0.
    Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
    Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

##### Задание

    На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)
    Реализуйте два алгоритма рекомендаций:

    сортировка просмотренных id по популярности (частота появления в просмотренных),
    сортировка просмотренных id по покупаемости (частота появления в покупках).

In [4]:
import collections

In [5]:
viewed = sessions.viewed.values
bought = sessions.bought.fillna(-1).values

In [6]:
from collections import Counter

In [7]:
viewed_dic = []
bought_dic = []
viewed_cnt = []
bought_cnt = []

viewed_dic_all = []
bought_dic_all = []
for idx, item  in enumerate(viewed):
    # инициализируем упорядоченные словари
    viewed_dic.append(collections.OrderedDict())
    viewed_cnt.append(collections.Counter())
    # парсим айдишники сессии как строки
    viewed[idx] = viewed[idx].split(",")
    # заполняем словарь и считаем количество вхождений
    viewed_dic_all += viewed[idx]
    for n,i in enumerate(viewed[idx]):
        try:
            count = viewed_dic[idx][i]
            viewed_dic[idx].update({i:count + 1})
        except KeyError:
            viewed_dic[idx].update({i:1})

In [8]:
viewed_dic[7]

OrderedDict([('59', 1),
             ('60', 2),
             ('61', 2),
             ('62', 1),
             ('63', 1),
             ('64', 1),
             ('65', 1),
             ('66', 1),
             ('67', 2),
             ('68', 1)])

In [9]:
for idx, item  in enumerate(bought):
    bought_dic.append(collections.OrderedDict())
    bought_cnt.append(collections.Counter())
    if(bought[idx] != -1):
        bought[idx] = bought[idx].split(",")
        bought_dic_all += bought[idx]
        for n,i in enumerate(bought[idx]):
            if(bought[idx] != -1):   
                try:
                    count = bought_dic[idx][i]
                    bought_dic[idx].update({i:count + 1})
                except KeyError:
                    bought_dic[idx].update({i:1})
            else:
                bought_dic[idx].update({-1:0})
    else:
        bought_dic[idx].update({-1:0})
        bought_dic_all += ['-1']
        continue

In [10]:
bought_dic[7]

OrderedDict([('67', 1), ('60', 1), ('63', 1)])

In [11]:
viewed_dic_all = collections.Counter(viewed_dic_all)
bought_dic_all = collections.Counter(bought_dic_all)

In [12]:
print(viewed_dic_all.most_common(5))
print(bought_dic_all.most_common(5))

[('73', 677), ('158', 641), ('204', 396), ('262', 387), ('162', 318)]
[('-1', 46392), ('158', 14), ('204', 12), ('3324', 11), ('73', 11)]


In [13]:
k1_viewed = []
for idx, item  in enumerate(viewed_dic):
    # заполняем частотами из словаря
    for key, value in viewed_dic[idx].items():
        viewed_dic[idx][key] = viewed_dic_all[key]   
    # получаем значение для топ-1 по просмотрам    
    val_t1_v = collections.Counter(viewed_dic[idx]).most_common(1)[0][1]
    #присваиваем для k1
    for key, value in viewed_dic[idx].items():
        if(value == val_t1_v):
            k1_viewed.append(key)
            break

In [14]:
print(viewed_dic[7])
print(k1_viewed[7])
print(bought_dic[7])
print(bought_dic[1])

OrderedDict([('59', 1), ('60', 2), ('61', 2), ('62', 1), ('63', 6), ('64', 3), ('65', 2), ('66', 2), ('67', 2), ('68', 2)])
63
OrderedDict([('67', 1), ('60', 1), ('63', 1)])
OrderedDict([(-1, 0)])


In [15]:
k1_bought = []
for idx, item  in enumerate(bought_dic):
    for key, value in bought_dic[idx].items():
        if(key != -1):
            bought_dic[idx][key] = bought_dic_all[key]
        else:
            bought_dic[idx][key] = 0
    # получаем значение для топ-1 по покупкам
        val_t1_b = collections.Counter(bought_dic[idx]).most_common(1)[0][1]
    for key, value in bought_dic[idx].items():
        if(value == val_t1_b):
            k1_bought.append(key)
            break

In [16]:
print(viewed_dic[7])
bought_dic[7]

OrderedDict([('59', 1), ('60', 2), ('61', 2), ('62', 1), ('63', 6), ('64', 3), ('65', 2), ('66', 2), ('67', 2), ('68', 2)])


OrderedDict([('67', 1), ('60', 1), ('63', 1)])

In [17]:
print(collections.Counter(viewed_dic[7]).most_common(5))

[('63', 6), ('64', 3), ('60', 2), ('67', 2), ('61', 2)]


In [18]:
k5_viewed = []
k5_bought = []
for idx, item  in enumerate(viewed_dic):
    if(idx != 300000):
        # получаем значение топ-5 по просмотрам
        val_t5_v = collections.Counter(viewed_dic[idx]).most_common(10)
        a = []
        i = 0; z = 0
        while i < 10:
            try:
                #if(idx == 7):
                #    print("t5=",val_t5_v[z],"i=",i)
                temp = val_t5_v[z]
                #присваиваем для k5
                for key, value in viewed_dic[idx].items():        
                    if(value == temp[1]):
                        if(len(a) !=5):
                            try:
                                a.index(key)
                                #if(idx == 7):  
                                    #print(key,value)
                                    #print(len(a))
                                continue
                            except ValueError:
                                #if(idx == 7):
                                #    print("i =",i,"; val_t5_v = ",temp[0])
                                a.append(key)
                                z +=1
                                i = 0
                            break
                        else:
                            break
            except IndexError:
                break
            i += 1
        k5_viewed.append(a)
        #if(idx == 7):
        #    print(k5_viewed)
    else:
        break

In [19]:
for idx, item  in enumerate(bought_dic):
    if(idx != 3000000):
        # получаем значение топ-5 по просмотрам
        val_t5_b = collections.Counter(bought_dic[idx]).most_common(10)
        a = []
        i = 0; z = 0
        while i < 10:
            try:
                #if(idx == 7):
                #    print("t5=",val_t5_v[z],"i=",i)
                temp = val_t5_b[z]
                #присваиваем для k5
                for key, value in bought_dic[idx].items():        
                    if(value == temp[1]):
                        if(len(a) !=5):
                            try:
                                a.index(key)
                                #if(idx == 7):  
                                    #print(key,value)
                                    #print(len(a))
                                continue
                            except ValueError:
                                #if(idx == 7):
                                #    print("i =",i,"; val_t5_v = ",temp[0])
                                a.append(key)
                                z +=1
                                i = 0
                            break
                        else:
                            break
            except IndexError:
                break
            i += 1
        k5_bought.append(a)
        #if(idx == 7):
        #    print(k5_viewed)
    else:
        break

In [20]:
def print_f(i):
    print(viewed_dic[i], '\nk1=',k1_viewed[i],'\nk5=', k5_viewed[i])
    print(bought_dic[i], '\nk1=',k1_bought[i],'\nk5=', k5_bought[i])
print_f(20)
viewed_dic[0]

OrderedDict([('206', 14), ('207', 37)]) 
k1= 207 
k5= ['207', '206']
OrderedDict([(-1, 0)]) 
k1= -1 
k5= [-1]


OrderedDict([('0', 6), ('1', 6), ('2', 9), ('3', 7), ('4', 11), ('5', 4)])

Прохожусь циклом по купленным из обучающей выборки и выбираю только те, где не пустая строка(т.е. есть хотя бы одна покупка) и получаю лист из всех купленных товаров для каждой сессии. 

Для каждого листа из предыдущего шаг считаю num = len(np.intersect1d(array, train_v)), где array количество купленных товаров в сессии и train_v - топ1 или топ 5 полученный выше. Далее получаю precision = num/len(top), где top это топ1 или топ5. И для recall = num/len(array)

#### AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5

$$Recal@k = {\frac{({Купленное-из-рекомендованного}  \cap  {top-k})}{Количество-рекомендаций(top-k)}}$$

$$Precision@k = \frac{({Купленное-из-рекомендованного}  \cap  {top-k})}{{количество-покупок}}$$

    Precision@k - Какая доля отрекомендаций покупается.
    Recal@k - Какая доля от того, что пользователи покупает в среднем рекомендуется нами и оказывается в покупках.

bought - количество товаров.

In [21]:
bought_cnt = []
tmp = []
k1_viewed_f = []
k5_viewed_f = []
k1_bought_f = []
k5_bought_f = []
for i,v in enumerate(bought):
    if(v != -1):
        k1_viewed_f.append([k1_viewed[i]])
        k5_viewed_f.append(k5_viewed[i])
        k1_bought_f.append([k1_bought[i]])
        k5_bought_f.append(k5_bought[i])
        tmp +=[bought[i]]
        bought_cnt += [len(v)]
bought = tmp

    Рекомендации по частоте просмотров товаров - качество на обучающей выборке:
    Рекомендации по частоте просмотров товаров - качество на тестовой выборке:
    Рекомендации по частоте покупок - качество на обучающей выборке:
    Рекомендации по частоте покупок - качество на тестовой выборке выборке:

### Рекомендации по частоте просмотров товаров - качество на обучающей выборке:


In [22]:
RecalA1v_train,PrecisionA1v_train,RecalA5v_train,PrecisionA5v_train = [], [], [], []

In [23]:
num = 0
for i,v in enumerate(bought):
    num_1 = len(np.intersect1d(np.array(bought[i]),np.array(k1_viewed_f[i]))) + 0.0
    RecalA1v_train.append((num_1/(len(bought[i]))))
    PrecisionA1v_train.append(num_1/1)
   

    num_5 = len(np.intersect1d(np.array(bought[i]),np.array(k5_viewed_f[i]))) + 0.0
    RecalA5v_train.append(num_5/(len(bought[i])))
    PrecisionA5v_train.append(num_5/5)
    
       

In [24]:
def write_answer_string_to_file(answer, filename):
    with open(filename, 'w') as f_out:
        f_out.write(answer)

In [25]:
print(round(mean(RecalA1v_train),2),
      round(mean(PrecisionA1v_train),2),
      round(mean(RecalA5v_train),2),
      round(mean(PrecisionA5v_train),2))

0.44 0.51 0.82 0.21


In [26]:
write_answer_string_to_file('0.44, 0.51, 0.82, 0.45','recomend_1.txt')

### Рекомендации по частоте покупок - качество на обучающей выборке:

In [27]:
RecalA1b_train,PrecisionA1b_train,RecalA5b_train,PrecisionA5b_train = [], [], [], []

In [28]:
num = 0
for i,v in enumerate(bought):
    num_1 = len(np.intersect1d(np.array(bought[i]),np.array(k1_bought_f[i]))) + 0.0
    PrecisionA1b_train.append(num_1/1)
    RecalA1b_train.append((num_1/(len(bought[i]))))

    num_5 = len(np.intersect1d(np.array(bought[i]),np.array(k5_bought_f[i]))) + 0.0
    PrecisionA5b_train.append(num_5/(len(k5_bought_f[i])))
    RecalA5b_train.append(num_5/5)

In [29]:
print(round(mean(RecalA1b_train),2),
      round(mean(PrecisionA1b_train),2),
      round(mean(RecalA5b_train),2),
      round(mean(PrecisionA5b_train),2))

0.85 1.0 0.29 1.0


In [30]:
write_answer_string_to_file('0.85, 1.0, 1.0, 1.0','recomend_3.txt')